In [1]:
!pip install rasterio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 42.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


**Calculate IoU,Overlap Percentage,Commission Error and Omission Errors**

In [3]:
import numpy as np
from osgeo import gdal

# Load raster as numpy array function
def load_raster_as_array(file_path):
    dataset = gdal.Open(file_path)
    if dataset is None:
        print(f"Could not open file {file_path}")
        return None
    band = dataset.GetRasterBand(1)
    array = band.ReadAsArray()
    array = np.where(array == 0, 0, array)  # Replace NoData values with 0
    return array

# Paths to your maps
benchmark_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/waterExtent_arcgis0_1.tif'
predicted_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/Flooding_Area_NDWI_S2_Resample.tif'# SENTINEL 2

# Load maps as numpy arrays
predicted_map = load_raster_as_array(predicted_map_path)
benchmark_map = load_raster_as_array(benchmark_map_path)

# Adjust both images to the smallest common shape
min_rows = min(predicted_map.shape[0], benchmark_map.shape[0])
min_cols = min(predicted_map.shape[1], benchmark_map.shape[1])

# Crop both maps to the smallest shape
predicted_map_cropped = predicted_map[:min_rows, :min_cols]
benchmark_map_cropped = benchmark_map[:min_rows, :min_cols]


# Check that both maps have the same dimensions
if predicted_map.shape != benchmark_map.shape:
    raise ValueError("The predicted map and benchmark map must have the same dimensions.")

# Binarize maps
predicted_flood_map = np.where(predicted_map >= 1, 1, 0)
benchmark_flood_map = np.where(benchmark_map >= 1, 1, 0)

print(f"SENTINEL-2:")

# IoU Calculation
intersection = np.logical_and(predicted_flood_map, benchmark_flood_map).sum()
union = np.logical_or(predicted_flood_map, benchmark_flood_map).sum()
iou = intersection / union if union != 0 else 0
print(f"IoU: {iou:.4f}")

# Overlap Percentage Calculation
overlap_percentage = (intersection / benchmark_flood_map.sum()) * 100 if benchmark_flood_map.sum() != 0 else 0
print(f"Overlap Percentage: {overlap_percentage:.2f}%")

# Commission Error: False Positives / (True Positives + False Positives)
false_positives = np.logical_and(predicted_flood_map == 1, benchmark_flood_map == 0).sum()
true_positives = intersection
commission_error = false_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
print(f"Commission Error: {commission_error* 100:.2f}%")


# Omission Error: False Negatives / (True Positives + False Negatives)
false_negatives = np.logical_and(predicted_flood_map == 0, benchmark_flood_map == 1).sum()
omission_error = false_negatives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
print(f"Omission Error: {omission_error* 100:.2f}%")


SENTINEL-2:
IoU: 0.4377
Overlap Percentage: 57.26%
Commission Error: 34.99%
Omission Error: 42.74%


In [4]:
#Code for SENTINEL-1
# Paths to your maps
benchmark_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/waterExtent_arcgis0_1.tif'
predicted_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/Flooding_Area_VH_S1_Resample.tif'  #SENTINEL 1

# Load maps as numpy arrays
predicted_map = load_raster_as_array(predicted_map_path)
benchmark_map = load_raster_as_array(benchmark_map_path)

# Adjust both images to the smallest common shape
min_rows = min(predicted_map.shape[0], benchmark_map.shape[0])
min_cols = min(predicted_map.shape[1], benchmark_map.shape[1])

# Crop both maps to the smallest shape
predicted_map_cropped = predicted_map[:min_rows, :min_cols]
benchmark_map_cropped = benchmark_map[:min_rows, :min_cols]


# Check that both maps have the same dimensions
if predicted_map.shape != benchmark_map.shape:
    raise ValueError("The predicted map and benchmark map must have the same dimensions.")

# Binarize maps
predicted_flood_map = np.where(predicted_map >= 1, 1, 0)
benchmark_flood_map = np.where(benchmark_map >= 1, 1, 0)

print(f"SENTINEL-1:")

# IoU Calculation
intersection = np.logical_and(predicted_flood_map, benchmark_flood_map).sum()
union = np.logical_or(predicted_flood_map, benchmark_flood_map).sum()
iou = intersection / union if union != 0 else 0
print(f"IoU: {iou:.4f}")

# Overlap Percentage Calculation
overlap_percentage = (intersection / benchmark_flood_map.sum()) * 100 if benchmark_flood_map.sum() != 0 else 0
print(f"Overlap Percentage: {overlap_percentage:.2f}%")

# Commission Error: False Positives / (True Positives + False Positives)
false_positives = np.logical_and(predicted_flood_map == 1, benchmark_flood_map == 0).sum()
true_positives = intersection
commission_error = false_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
print(f"Commission Error: {commission_error* 100:.2f}%")


# Omission Error: False Negatives / (True Positives + False Negatives)
false_negatives = np.logical_and(predicted_flood_map == 0, benchmark_flood_map == 1).sum()
omission_error = false_negatives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
print(f"Omission Error: {omission_error* 100:.2f}%")


SENTINEL-1:
IoU: 0.2181
Overlap Percentage: 35.05%
Commission Error: 63.41%
Omission Error: 64.95%


In [5]:
#Code for LANDSAT
# Paths to your maps
benchmark_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/waterExtent_arcgis0_1.tif'
predicted_map_path = '/content/gdrive/MyDrive/Export_TIFF_FLOOD_TETOUAN/Flooding_Area_NDWI_LSAT_Resample.tif'#LANDSAT

# Load maps as numpy arrays
predicted_map = load_raster_as_array(predicted_map_path)
benchmark_map = load_raster_as_array(benchmark_map_path)

# Adjust both images to the smallest common shape
min_rows = min(predicted_map.shape[0], benchmark_map.shape[0])
min_cols = min(predicted_map.shape[1], benchmark_map.shape[1])

# Crop both maps to the smallest shape
predicted_map_cropped = predicted_map[:min_rows, :min_cols]
benchmark_map_cropped = benchmark_map[:min_rows, :min_cols]


# Check that both maps have the same dimensions
if predicted_map_cropped.shape != benchmark_map_cropped.shape:
    raise ValueError("The predicted map and benchmark map must have the same dimensions.")

# Binarize maps
predicted_flood_map = np.where(predicted_map_cropped >= 1, 1, 0)
benchmark_flood_map = np.where(benchmark_map_cropped >= 1, 1, 0)

print(f"LANDSAT:")

# IoU Calculation
intersection = np.logical_and(predicted_flood_map, benchmark_flood_map).sum()
union = np.logical_or(predicted_flood_map, benchmark_flood_map).sum()
iou = intersection / union if union != 0 else 0
print(f"IoU: {iou:.4f}")

# Overlap Percentage Calculation
overlap_percentage = (intersection / benchmark_flood_map.sum()) * 100 if benchmark_flood_map.sum() != 0 else 0
print(f"Overlap Percentage: {overlap_percentage:.2f}%")

# Commission Error: False Positives / (True Positives + False Positives)
false_positives = np.logical_and(predicted_flood_map == 1, benchmark_flood_map == 0).sum()
true_positives = intersection
commission_error = false_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
print(f"Commission Error: {commission_error* 100:.2f}%")


# Omission Error: False Negatives / (True Positives + False Negatives)
false_negatives = np.logical_and(predicted_flood_map == 0, benchmark_flood_map == 1).sum()
omission_error = false_negatives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
print(f"Omission Error: {omission_error* 100:.2f}%")


LANDSAT:
IoU: 0.3347
Overlap Percentage: 58.95%
Commission Error: 56.35%
Omission Error: 41.05%


**Overlap Percentage with ArcGis**

In [6]:
intersected_Area_s1= 3081; #from .tif in Arcgis;
intersected_Area_s2=5033; #from .tif in Arcgis;
intersected_Area_ls=5315; #from .tif in Arcgis ;
Flood_area_arc=8790;

Overlap_Percentage_s1=(intersected_Area_s1/Flood_area_arc)*100;
Overlap_Percentage_s2=(intersected_Area_s2/Flood_area_arc)*100;
Overlap_Percentage_ls=(intersected_Area_ls/Flood_area_arc)*100;

print(f"Overlap Percentage sentinel-1:{Overlap_Percentage_s1:.2f}");
print(f"Overlap Percentage sentinel-2:{Overlap_Percentage_s2:.2f}");
print(f"Overlap Percentage landsat:{Overlap_Percentage_ls:.2f}");


Overlap Percentage sentinel-1:35.05
Overlap Percentage sentinel-2:57.26
Overlap Percentage landsat:60.47
